# Data Preparation

In [141]:
#Upload the data

import pandas as pd

df_skill_builder_data = pd.read_csv(
    "skill_builder_data_corrected_collapsed (1).csv",
    encoding="latin1"
)



C:\Users\Richard Han\AppData\Local\Temp\ipykernel_8180\435537265.py:5: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_skill_builder_data = pd.read_csv(


In [119]:
#Only get necessary columns

df_clean = df_skill_builder_data[[
    "user_id",
    "template_id",
    "problem_id",
    "original",
    "order_id",
    "attempt_count",
    "answer_type",
    "answer_id",
    "answer_text",
    "correct",
    "school_id",
    "skill_id",
    "skill_name"
]].copy()

# Make sure dtypes are correct
df_clean["user_id"] = df_clean["user_id"].astype(str)
df_clean["template_id"] = df_clean["template_id"].astype(str)
df_clean["problem_id"] = df_clean["problem_id"].astype(str)
df_clean["order_id"] = df_clean["order_id"].astype(str)
df_clean["attempt_count"] = df_clean["attempt_count"].astype(str)
df_clean["answer_type"] = df_clean["answer_type"].astype(str)
df_clean["answer_id"] = df_clean["answer_id"].astype(str)
df_clean["answer_text"] = df_clean["answer_text"].astype(str)
df_clean["school_id"] = df_clean["school_id"].astype(str)
df_clean["skill_id"] = df_clean["skill_id"].astype(str)
df_clean["skill_name"] = df_clean["skill_name"].astype(str)
df_clean["correct"] = df_clean["correct"].astype(int)
df_clean["original"] = df_clean["original"].astype(int)

In [120]:
#Make sure all skill ids are integers because some appear as floats
df_clean['skill_id'] = df_clean['skill_id'].astype(str)

# remove trailing .0 ONLY when it exists
df_clean['skill_id'] = df_clean['skill_id'].str.replace(r'\.0$', '', regex=True)


In [121]:
#Filter to single skill id only
is_multi = df_clean['skill_id'].str.contains('_')
single_skill_df = df_clean[~is_multi]


In [122]:
#Delete rows with 'nan' values for skill_id
single_skill_df = single_skill_df[single_skill_df['skill_id'] != 'nan']


In [123]:
#See which skill_id has most rows
single_skill_df["skill_id"].value_counts(dropna=False).head(20)
single_skill_df["skill_id"].value_counts(dropna=False).head(50)

skill_id
311    24253
47     18739
277    12741
280    11334
312     8115
79      8068
279     7058
27      6590
18      6557
50      6117
77      6109
67      5547
325     5398
49      4895
310     4659
46      4434
278     4320
70      4263
61      4029
17      3890
11      3864
65      3256
81      3183
309     3072
76      3050
297     2978
86      2947
85      2900
83      2813
75      2576
51      2007
276     1970
82      1888
8       1859
58      1816
4       1804
368     1792
40      1769
294     1760
308     1706
94      1533
25      1524
301     1480
13      1451
101     1332
92      1301
303     1285
39      1149
54       951
322      949
Name: count, dtype: int64

In [124]:
#See how many unique template_id's the candidate skill_id has
single_skill_df[single_skill_df["skill_id"] == '311']["template_id"].nunique()


17

In [125]:
#See the skill name associated with skill 311.
single_skill_df[single_skill_df["skill_id"] == '311']["skill_name"].unique()

array(['Equation Solving Two or Fewer Steps'], dtype=object)

In [126]:
single_skill_df.columns

Index(['user_id', 'template_id', 'problem_id', 'original', 'order_id',
       'attempt_count', 'answer_type', 'answer_id', 'answer_text', 'correct',
       'school_id', 'skill_id', 'skill_name'],
      dtype='object')

In [127]:
#Define df just for skill_id = '311'
df_311 = single_skill_df[single_skill_df['skill_id'] == '311'].copy()



In [128]:
df_311['answer_type'].unique()

array(['algebra', 'choose_1'], dtype=object)

In [129]:
#Define datasets that have MC items and Free-response items respectively
# MC = 3PL item model
mc_items = df_311[df_311['answer_type'] == 'choose_1']

# Free-response = 2PL item model
fr_items = df_311[df_311['answer_type'] == 'algebra']


In [130]:
mc_items.shape, fr_items.shape

((5293, 13), (18960, 13))

In [131]:
fr_items['template_id'].nunique()

15

In [15]:
#Choose to go with free response items dataset


In [132]:
#Collapse to one row per student-item
df_main = fr_items[fr_items["original"] == 1].copy()
# sort so earliest log for each (user, template) comes first
df_first = (
    df_main
    .sort_values(["user_id", "template_id", "order_id"])
    .groupby(["user_id", "template_id"], as_index=False)
    .first()   # first row within each group
)

# keep only what we need for IRT (+ school_id for DIF later)
df_irt = (
    df_first[["user_id", "template_id", "correct", "school_id"]]
    .rename(columns={
        "user_id": "student_id",
        "template_id": "item_id",
        "correct": "is_correct"
    })
)

df_irt["student_id"] = df_irt["student_id"].astype(str)
df_irt["item_id"]    = df_irt["item_id"].astype(str)
df_irt["is_correct"] = df_irt["is_correct"].astype(int)

# sanity check: should be exactly one row per student–item
print(df_irt.duplicated(["student_id", "item_id"]).sum())
print(df_irt.head())



0
  student_id item_id  is_correct school_id
0      51950   54581           1      2770
1      51950   55506           0      2770
2      51950   55509           1      2770
3      53167   54581           0        73
4      53167   54697           0        73


In [139]:
df_irt['student_id'].nunique()

959

In [133]:
df_irt.to_csv("df_irt.csv", index=False)